In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Определение архитектуры LeNet

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 входное изображение, 6 выходных каналов, 3х3 квадратная свертка
        # kernel 
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
    
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max-Pooling через (2, 2) окно
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Если размер окна - квадрат, то можно указывать только одно число
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [34]:
input = torch.randn(1, 1, 32, 32)

In [62]:
# Демонстрация использования автограда для обучения
output = net(input)
target = torch.randn(output.size())

# Обозначаем критерий ошибки
criterion = nn.MSELoss()
loss = criterion(output, target)

# Для наглядности обнуляем накопленные градиенты
net.zero_grad()
print(net.fc1.bias.grad)

# Проверяем, изменились ли градиенты
loss.backward()
print(net.fc1.bias.grad)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00, -2.3886e-03,  0.0000e+00,
         5.8390e-03, -3.4384e-03,  0.0000e+00, -4.2842e-04, -2.1810e-02,
        -4.4317e-03,  0.0000e+00,  0.0000e+00, -5.8346e-03,  3.1148e-03,
         0.0000e+00,  2.3394e-02,  0.0000e+00,  5.4328e-03,  1.1563e-02,
         1.9078e-02,  0.0000e+00, -5.7501e-03, -1.6843e-02, -7.8907e-03,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  8.2680

## Обновление весов

In [67]:
# Простая схема обновления весов

learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [ ]:
# Можно испольховать строенные оптимизаторы из модуля torch.optim
import torch.optim as optim

# Создаем оптимизатор
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Добавляем в обучающий цикл
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()